In [6]:
# Personal Finance Database Import - Complete Data Migration
# This notebook will import all data from data.xlsx into the personal_finance.db

import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime, date
import os
import shutil
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('data_import.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("🚀 Personal Finance Database Import Tool")
print("=" * 60)
print(f"Started at: {datetime.now()}")
print("=" * 60)

🚀 Personal Finance Database Import Tool
Started at: 2025-06-30 14:25:29.458689


In [7]:
# =============================================================================
# STEP 1: DATABASE CONNECTION AND BACKUP
# =============================================================================

def create_backup():
    """Create a backup of the current database"""
    try:
        db_path = 'personal_finance.db'
        if os.path.exists(db_path):
            backup_path = f'personal_finance_backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.db'
            shutil.copy2(db_path, backup_path)
            logger.info(f"✅ Database backup created: {backup_path}")
            return backup_path
        else:
            logger.info("ℹ️ No existing database found - fresh import")
            return None
    except Exception as e:
        logger.error(f"❌ Error creating backup: {e}")
        raise

def connect_database():
    """Connect to the database and return connection"""
    try:
        conn = sqlite3.connect('personal_finance.db')
        logger.info("✅ Connected to personal_finance.db")
        return conn
    except Exception as e:
        logger.error(f"❌ Error connecting to database: {e}")
        raise

# Create backup and connect
backup_file = create_backup()
conn = connect_database()

2025-06-30 14:25:29,472 - INFO - ✅ Database backup created: personal_finance_backup_20250630_142529.db
2025-06-30 14:25:29,473 - INFO - ✅ Connected to personal_finance.db


In [8]:
# =============================================================================
# STEP 2: ANALYZE DATABASE STRUCTURE
# =============================================================================

def analyze_database_structure(conn):
    """Analyze the current database structure"""
    cursor = conn.cursor()
    
    print("\n📊 DATABASE STRUCTURE ANALYSIS")
    print("-" * 40)
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    structure_info = {}
    
    for table in tables:
        table_name = table[0]
        print(f"\n🔍 Table: {table_name}")
        
        # Get table info
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        # Get row count
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        
        print(f"   Rows: {row_count}")
        print("   Columns:")
        
        table_columns = []
        for col in columns:
            col_info = {
                'name': col[1],
                'type': col[2],
                'not_null': bool(col[3]),
                'default': col[4],
                'primary_key': bool(col[5])
            }
            table_columns.append(col_info)
            
            required = "REQUIRED" if col_info['not_null'] else "optional"
            pk = " (PK)" if col_info['primary_key'] else ""
            print(f"     - {col_info['name']}: {col_info['type']} {required}{pk}")
        
        structure_info[table_name] = {
            'columns': table_columns,
            'row_count': row_count
        }
    
    return structure_info

# Analyze current structure
db_structure = analyze_database_structure(conn)


📊 DATABASE STRUCTURE ANALYSIS
----------------------------------------

🔍 Table: transactions
   Rows: 5878
   Columns:
     - id: INTEGER optional (PK)
     - account_name: TEXT REQUIRED
     - date: DATE REQUIRED
     - description: TEXT REQUIRED
     - amount: DECIMAL(10,2) REQUIRED
     - sub_category: TEXT optional
     - category: TEXT REQUIRED
     - type: TEXT REQUIRED
     - owner: TEXT REQUIRED
     - is_business: BOOLEAN optional
     - debt_payment_id: INTEGER optional
     - created_at: TIMESTAMP optional
     - updated_at: TIMESTAMP optional
     - is_active: BOOLEAN optional

🔍 Table: sqlite_sequence
   Rows: 5
   Columns:
     - name:  optional
     - seq:  optional

🔍 Table: budget_templates
   Rows: 23
   Columns:
     - id: INTEGER optional (PK)
     - category: TEXT REQUIRED
     - budget_amount: DECIMAL(10,2) REQUIRED
     - notes: TEXT optional
     - is_active: BOOLEAN optional
     - created_at: TIMESTAMP optional
     - updated_at: TIMESTAMP optional

🔍 Table:

In [9]:
# =============================================================================
# STEP 3: LOAD AND ANALYZE EXCEL DATA
# =============================================================================

def load_excel_data():
    """Load and analyze the Excel data"""
    try:
        print("\n📈 EXCEL DATA ANALYSIS")
        print("-" * 40)
        
        # Load the Excel file
        excel_file = 'data.xlsm'
        if not os.path.exists(excel_file):
            raise FileNotFoundError(f"Excel file not found: {excel_file}")
        
        # Get all sheet names
        xl_file = pd.ExcelFile(excel_file)
        sheet_names = xl_file.sheet_names
        print(f"📋 Found {len(sheet_names)} sheets: {sheet_names}")
        
        sheets_data = {}
        
        for sheet_name in sheet_names:
            print(f"\n🔍 Analyzing sheet: {sheet_name}")
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            
            print(f"   Rows: {len(df)}")
            print(f"   Columns: {len(df.columns)}")
            print("   Column names:")
            for col in df.columns:
                print(f"     - {col}")
            
            # Check for missing values
            missing_data = df.isnull().sum()
            if missing_data.sum() > 0:
                print("   Missing values:")
                for col, missing_count in missing_data.items():
                    if missing_count > 0:
                        print(f"     - {col}: {missing_count} missing")
            
            # Sample data
            print("   Sample data (first 3 rows):")
            print(df.head(3).to_string(index=False))
            
            sheets_data[sheet_name] = df
        
        return sheets_data
        
    except Exception as e:
        logger.error(f"❌ Error loading Excel data: {e}")
        raise

# Load Excel data
excel_data = load_excel_data()


📈 EXCEL DATA ANALYSIS
----------------------------------------
📋 Found 1 sheets: ['Transactions']

🔍 Analyzing sheet: Transactions
   Rows: 6044
   Columns: 8
   Column names:
     - Account Name
     - Date
     - Description
     - Amount
     - Sub Category
     - Category
     - Type
     - Owner
   Missing values:
     - Date: 1 missing
     - Description: 21 missing
     - Sub Category: 1 missing
     - Category: 1 missing
     - Type: 1 missing
     - Owner: 1 missing
   Sample data (first 3 rows):
Account Name       Date Description Amount  Sub Category        Category  Type    Owner
     Venture 2025-06-28        HEB    53.8     Groceries Living Expenses Needs Suricata
     Venture 2025-06-28     Peacock   8.65 Subscriptions   Subscriptions Wants Suricata
     Venture 2025-06-27        HEB   42.87     Groceries Living Expenses Needs Suricata


In [10]:
# =============================================================================
# STEP 4: CLEAR EXISTING DATA
# =============================================================================

def clear_existing_data(conn):
    """Clear existing data from all tables"""
    cursor = conn.cursor()
    
    print("\n🧹 CLEARING EXISTING DATA")
    print("-" * 40)
    
    tables_to_clear = [
        'transactions',
        'budget_templates', 
        'unexpected_expenses',
        'monthly_budgets',
        'debt_accounts',
        'debt_payments',
        'business_revenue'
    ]
    
    for table in tables_to_clear:
        try:
            # Check if table exists
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table,))
            if cursor.fetchone():
                # Get count before deletion
                cursor.execute(f"SELECT COUNT(*) FROM {table}")
                count_before = cursor.fetchone()[0]
                
                # Clear the table
                cursor.execute(f"DELETE FROM {table}")
                rows_deleted = cursor.rowcount
                
                # Reset auto-increment if it's the transactions table
                if table == 'transactions':
                    cursor.execute("DELETE FROM sqlite_sequence WHERE name='transactions'")
                
                print(f"✅ Cleared {table}: {rows_deleted} rows deleted")
                logger.info(f"Cleared {table}: {rows_deleted} rows deleted")
            else:
                print(f"⚠️ Table {table} not found")
        except Exception as e:
            logger.error(f"❌ Error clearing {table}: {e}")
            print(f"❌ Error clearing {table}: {e}")
    
    conn.commit()
    print("✅ All existing data cleared")

# Clear existing data
clear_existing_data(conn)

2025-06-30 14:25:29,961 - INFO - Cleared transactions: 5878 rows deleted
2025-06-30 14:25:29,962 - INFO - Cleared budget_templates: 23 rows deleted
2025-06-30 14:25:29,963 - INFO - Cleared unexpected_expenses: 1 rows deleted
2025-06-30 14:25:29,963 - INFO - Cleared monthly_budgets: 0 rows deleted
2025-06-30 14:25:29,964 - INFO - Cleared debt_accounts: 5 rows deleted
2025-06-30 14:25:29,964 - INFO - Cleared debt_payments: 2 rows deleted
2025-06-30 14:25:29,965 - INFO - Cleared business_revenue: 0 rows deleted



🧹 CLEARING EXISTING DATA
----------------------------------------
✅ Cleared transactions: 5878 rows deleted
✅ Cleared budget_templates: 23 rows deleted
✅ Cleared unexpected_expenses: 1 rows deleted
✅ Cleared monthly_budgets: 0 rows deleted
✅ Cleared debt_accounts: 5 rows deleted
✅ Cleared debt_payments: 2 rows deleted
✅ Cleared business_revenue: 0 rows deleted
✅ All existing data cleared


In [11]:
# =============================================================================
# STEP 5: DYNAMIC DATA MAPPING BASED ON ACTUAL EXCEL STRUCTURE
# =============================================================================

def detect_excel_structure(excel_data):
    """Automatically detect Excel structure and create mapping"""
    
    print("\n🔍 DETECTING EXCEL STRUCTURE")
    print("-" * 40)
    
    # Find the transactions sheet
    transactions_sheet = None
    if 'Transactions' in excel_data:
        transactions_sheet = 'Transactions'
        print(f"✅ Found transactions sheet: {transactions_sheet}")
    elif 'Sheet1' in excel_data:
        transactions_sheet = 'Sheet1'
        print(f"✅ Found default sheet: {transactions_sheet}")
    else:
        # Use the first sheet
        transactions_sheet = list(excel_data.keys())[0]
        print(f"⚠️ Using first available sheet: {transactions_sheet}")
    
    df = excel_data[transactions_sheet]
    excel_columns = list(df.columns)
    print(f"📋 Available columns: {excel_columns}")
    
    # Smart column detection
    column_mapping = {}
    
    # Date column detection
    date_patterns = ['date', 'Date', 'DATE', 'transaction_date', 'Transaction Date']
    for pattern in date_patterns:
        if pattern in excel_columns:
            column_mapping['date'] = pattern
            break
    
    # Description column detection
    desc_patterns = ['description', 'Description', 'DESC', 'details', 'Details', 'transaction', 'Transaction']
    for pattern in desc_patterns:
        if pattern in excel_columns:
            column_mapping['description'] = pattern
            break
    
    # Amount column detection
    amount_patterns = ['amount', 'Amount', 'AMOUNT', 'value', 'Value', 'transaction_amount']
    for pattern in amount_patterns:
        if pattern in excel_columns:
            column_mapping['amount'] = pattern
            break
    
    # Category column detection
    category_patterns = ['category', 'Category', 'CATEGORY', 'type', 'Type']
    for pattern in category_patterns:
        if pattern in excel_columns:
            column_mapping['category'] = pattern
            break
    
    # Sub-category detection
    subcategory_patterns = ['sub_category', 'Sub Category', 'subcategory', 'Subcategory', 'sub category']
    for pattern in subcategory_patterns:
        if pattern in excel_columns:
            column_mapping['sub_category'] = pattern
            break
    
    # Transaction type detection
    type_patterns = ['type', 'Type', 'transaction_type', 'Transaction Type', 'expense_type']
    for pattern in type_patterns:
        if pattern in excel_columns:
            column_mapping['type'] = pattern
            break
    
    # Owner detection
    owner_patterns = ['owner', 'Owner', 'OWNER', 'person', 'Person', 'user', 'User']
    for pattern in owner_patterns:
        if pattern in excel_columns:
            column_mapping['owner'] = pattern
            break
    
    # Account detection
    account_patterns = ['account', 'Account', 'ACCOUNT', 'account_name', 'Account Name', 'bank', 'Bank']
    for pattern in account_patterns:
        if pattern in excel_columns:
            column_mapping['account_name'] = pattern
            break
    
    # Business flag detection
    business_patterns = ['business', 'Business', 'is_business', 'Is Business', 'girasoul', 'Girasoul']
    for pattern in business_patterns:
        if pattern in excel_columns:
            column_mapping['is_business'] = pattern
            break
    
    print(f"\n📊 Detected column mapping:")
    for db_col, excel_col in column_mapping.items():
        print(f"   {db_col} -> {excel_col}")
    
    return transactions_sheet, column_mapping

def create_data_mapping(excel_data):
    """Create mapping between Excel columns and database columns"""
    
    # Detect structure automatically
    sheet_name, column_mapping = detect_excel_structure(excel_data)
    
    mapping = {
        'transactions': {
            'excel_sheet': sheet_name,
            'column_mapping': column_mapping,
            'transformations': {
                'date': lambda x: pd.to_datetime(x).date() if pd.notna(x) else None,
                'amount': lambda x: float(x) if pd.notna(x) else 0.0,
                'is_business': lambda x: bool(x) if pd.notna(x) and str(x).lower() in ['true', '1', 'yes', 'y'] else False
            }
        }
    }
    
    return mapping

def transform_data_for_import(excel_data, mapping):
    """Transform Excel data for database import"""
    
    print("\n🔄 DATA TRANSFORMATION")
    print("-" * 40)
    
    transformed_data = {}
    
    for table_name, config in mapping.items():
        print(f"\n📋 Transforming data for: {table_name}")
        
        sheet_name = config['excel_sheet']
        if sheet_name not in excel_data:
            print(f"⚠️ Sheet {sheet_name} not found in Excel data")
            continue
        
        df = excel_data[sheet_name].copy()
        print(f"   Source rows: {len(df)}")
        
        # Apply column mapping
        column_mapping = config['column_mapping']
        available_columns = [col for col in column_mapping.values() if col in df.columns]
        
        if not available_columns:
            print(f"   ❌ No mapped columns found in Excel data")
            continue
        
        print(f"   ✅ Available columns: {available_columns}")
        
        # Create new DataFrame with mapped columns
        df_mapped = pd.DataFrame()
        
        for db_col, excel_col in column_mapping.items():
            if excel_col in df.columns:
                df_mapped[db_col] = df[excel_col]
                print(f"   ✅ Mapped: {excel_col} -> {db_col}")
        
        # Apply transformations
        transformations = config.get('transformations', {})
        for col, transform_func in transformations.items():
            if col in df_mapped.columns:
                try:
                    df_mapped[col] = df_mapped[col].apply(transform_func)
                    print(f"   ✅ Transformed column: {col}")
                except Exception as e:
                    print(f"   ❌ Error transforming {col}: {e}")
        
        # Add required fields that might be missing
        if table_name == 'transactions':
            # Add timestamps if not present
            if 'created_at' not in df_mapped.columns:
                df_mapped['created_at'] = datetime.utcnow().isoformat()
            if 'updated_at' not in df_mapped.columns:
                df_mapped['updated_at'] = datetime.utcnow().isoformat()
            if 'is_active' not in df_mapped.columns:
                df_mapped['is_active'] = 1
            
            # Ensure required fields have defaults
            if 'debt_payment_id' not in df_mapped.columns:
                df_mapped['debt_payment_id'] = None
        
        # Remove rows with missing critical data
        initial_count = len(df_mapped)
        required_fields = ['date', 'description', 'amount']
        available_required = [field for field in required_fields if field in df_mapped.columns]
        
        if available_required:
            df_mapped = df_mapped.dropna(subset=available_required)
            final_count = len(df_mapped)
            
            if initial_count != final_count:
                print(f"   ⚠️ Dropped {initial_count - final_count} rows with missing required data")
        
        print(f"   ✅ Final rows for import: {len(df_mapped)}")
        
        # Show sample of transformed data
        if len(df_mapped) > 0:
            print(f"   📊 Sample transformed data:")
            print(df_mapped.head(2).to_string(index=False))
        
        transformed_data[table_name] = df_mapped
    
    return transformed_data

In [12]:
# =============================================================================
# STEP 6: IMPORT DATA WITH PROGRESS TRACKING
# =============================================================================

def import_data_with_progress(conn, transformed_data):
    """Import data with detailed progress tracking"""
    
    print("\n📥 IMPORTING DATA")
    print("-" * 40)
    
    cursor = conn.cursor()
    import_results = {}
    
    for table_name, df in transformed_data.items():
        print(f"\n📋 Importing to {table_name}")
        
        try:
            total_rows = len(df)
            if total_rows == 0:
                print(f"   ⚠️ No data to import for {table_name}")
                import_results[table_name] = {'total': 0, 'imported': 0, 'errors': 0}
                continue
                
            batch_size = 100
            imported_count = 0
            error_count = 0
            
            print(f"   Total rows to import: {total_rows}")
            print(f"   Columns to import: {list(df.columns)}")
            
            # Import in batches
            for start_idx in range(0, total_rows, batch_size):
                end_idx = min(start_idx + batch_size, total_rows)
                batch_df = df.iloc[start_idx:end_idx]
                
                try:
                    # Insert batch
                    batch_df.to_sql(table_name, conn, if_exists='append', index=False)
                    imported_count += len(batch_df)
                    
                    # Progress update
                    progress = (imported_count / total_rows) * 100
                    print(f"   Progress: {imported_count}/{total_rows} ({progress:.1f}%)")
                    
                    # Commit every 10 batches
                    if (start_idx // batch_size) % 10 == 0:
                        conn.commit()
                    
                except Exception as e:
                    error_count += len(batch_df)
                    logger.error(f"Error importing batch {start_idx}-{end_idx} to {table_name}: {e}")
                    print(f"   ❌ Error in batch {start_idx}-{end_idx}: {e}")
                    
                    # Try individual rows in failed batch
                    for idx, row in batch_df.iterrows():
                        try:
                            row_df = pd.DataFrame([row])
                            row_df.to_sql(table_name, conn, if_exists='append', index=False)
                            imported_count += 1
                            error_count -= 1
                        except Exception as row_error:
                            logger.error(f"Error importing row {idx}: {row_error}")
                            print(f"   ❌ Failed row {idx}: {row_error}")
            
            # Final commit
            conn.commit()
            
            # Final results
            print(f"   ✅ Import completed:")
            print(f"      - Successful: {imported_count}")
            print(f"      - Errors: {error_count}")
            print(f"      - Success rate: {(imported_count/total_rows*100):.1f}%")
            
            import_results[table_name] = {
                'total': total_rows,
                'imported': imported_count,
                'errors': error_count
            }
            
        except Exception as e:
            logger.error(f"❌ Error importing {table_name}: {e}")
            print(f"❌ Error importing {table_name}: {e}")
            import_results[table_name] = {
                'total': len(df) if 'df' in locals() else 0,
                'imported': 0,
                'errors': len(df) if 'df' in locals() else 0
            }
    
    return import_results


In [13]:
# =============================================================================
# STEP 7: EXECUTE IMPORT
# =============================================================================

print("\n🚀 EXECUTING IMPORT")
print("-" * 40)

try:
    # Create the mapping configuration
    mapping = create_data_mapping(excel_data)
    
    # Transform the Excel data
    transformed_data = transform_data_for_import(excel_data, mapping)
    
    # Import with progress tracking
    import_results = import_data_with_progress(conn, transformed_data)
    
    print("\n📊 IMPORT SUMMARY")
    print("-" * 40)
    for table, results in import_results.items():
        print(f"{table}:")
        print(f"  Total: {results['total']}")
        print(f"  Imported: {results['imported']}")
        print(f"  Errors: {results['errors']}")
        success_rate = (results['imported'] / results['total'] * 100) if results['total'] > 0 else 0
        print(f"  Success Rate: {success_rate:.1f}%")
    
except Exception as e:
    logger.error(f"❌ Error during import execution: {e}")
    print(f"❌ Error during import execution: {e}")
    import traceback
    traceback.print_exc()



🚀 EXECUTING IMPORT
----------------------------------------

🔍 DETECTING EXCEL STRUCTURE
----------------------------------------
✅ Found transactions sheet: Transactions
📋 Available columns: ['Account Name', 'Date', 'Description', 'Amount', 'Sub Category', 'Category', 'Type', 'Owner']

📊 Detected column mapping:
   date -> Date
   description -> Description
   amount -> Amount
   category -> Category
   sub_category -> Sub Category
   type -> Type
   owner -> Owner
   account_name -> Account Name

🔄 DATA TRANSFORMATION
----------------------------------------

📋 Transforming data for: transactions
   Source rows: 6044
   ✅ Available columns: ['Date', 'Description', 'Amount', 'Category', 'Sub Category', 'Type', 'Owner', 'Account Name']
   ✅ Mapped: Date -> date
   ✅ Mapped: Description -> description
   ✅ Mapped: Amount -> amount
   ✅ Mapped: Category -> category
   ✅ Mapped: Sub Category -> sub_category
   ✅ Mapped: Type -> type
   ✅ Mapped: Owner -> owner
   ✅ Mapped: Account Name -

C:\Users\ELCACAZ\AppData\Local\Temp\ipykernel_1348\3538824744.py:171: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df_mapped['created_at'] = datetime.utcnow().isoformat()
C:\Users\ELCACAZ\AppData\Local\Temp\ipykernel_1348\3538824744.py:173: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df_mapped['updated_at'] = datetime.utcnow().isoformat()


   Progress: 1500/6023 (24.9%)
   Progress: 1600/6023 (26.6%)
   Progress: 1700/6023 (28.2%)
   Progress: 1800/6023 (29.9%)
   Progress: 1900/6023 (31.5%)
   Progress: 2000/6023 (33.2%)
   Progress: 2100/6023 (34.9%)
   Progress: 2200/6023 (36.5%)
   Progress: 2300/6023 (38.2%)
   Progress: 2400/6023 (39.8%)
   Progress: 2500/6023 (41.5%)
   Progress: 2600/6023 (43.2%)
   Progress: 2700/6023 (44.8%)
   Progress: 2800/6023 (46.5%)
   Progress: 2900/6023 (48.1%)
   Progress: 3000/6023 (49.8%)
   Progress: 3100/6023 (51.5%)
   Progress: 3200/6023 (53.1%)
   Progress: 3300/6023 (54.8%)
   Progress: 3400/6023 (56.5%)
   Progress: 3500/6023 (58.1%)
   Progress: 3600/6023 (59.8%)
   Progress: 3700/6023 (61.4%)
   Progress: 3800/6023 (63.1%)
   Progress: 3900/6023 (64.8%)
   Progress: 4000/6023 (66.4%)
   Progress: 4100/6023 (68.1%)
   Progress: 4200/6023 (69.7%)
   Progress: 4300/6023 (71.4%)
   Progress: 4400/6023 (73.1%)
   Progress: 4500/6023 (74.7%)
   Progress: 4600/6023 (76.4%)
   Progr

In [14]:
# =============================================================================
# STEP 8: VALIDATION AND REPORTING
# =============================================================================

def validate_import(conn):
    """Validate the imported data"""
    print("\n✅ IMPORT VALIDATION")
    print("-" * 40)
    
    cursor = conn.cursor()
    validation_results = {}
    
    try:
        # Check transaction counts
        cursor.execute("SELECT COUNT(*) FROM transactions")
        transaction_count = cursor.fetchone()[0]
        print(f"📊 Total transactions imported: {transaction_count}")
        
        if transaction_count > 0:
            # Check date range
            cursor.execute("SELECT MIN(date), MAX(date) FROM transactions")
            date_range = cursor.fetchone()
            print(f"📅 Date range: {date_range[0]} to {date_range[1]}")
            
            # Check categories
            cursor.execute("SELECT COUNT(DISTINCT category) FROM transactions WHERE category IS NOT NULL")
            category_count = cursor.fetchone()[0]
            print(f"📂 Unique categories: {category_count}")
            
            # Check owners
            cursor.execute("SELECT COUNT(DISTINCT owner) FROM transactions WHERE owner IS NOT NULL")
            owner_count = cursor.fetchone()[0]
            print(f"👥 Unique owners: {owner_count}")
            
            # Check for missing data
            cursor.execute("SELECT COUNT(*) FROM transactions WHERE description IS NULL OR description = ''")
            missing_descriptions = cursor.fetchone()[0]
            print(f"⚠️ Missing descriptions: {missing_descriptions}")
            
            cursor.execute("SELECT COUNT(*) FROM transactions WHERE category IS NULL OR category = ''")
            missing_categories = cursor.fetchone()[0]
            print(f"⚠️ Missing categories: {missing_categories}")
            
            # Sample data check
            cursor.execute("SELECT * FROM transactions ORDER BY date DESC LIMIT 5")
            sample_data = cursor.fetchall()
            print(f"\n📋 Sample imported data (latest 5 transactions):")
            for row in sample_data:
                print(f"   {row[2]} | {row[3]} | ${row[4]} | {row[6]}")  # date, description, amount, category
            
            return {
                'total_transactions': transaction_count,
                'date_range': date_range,
                'categories': category_count,
                'owners': owner_count,
                'missing_descriptions': missing_descriptions,
                'missing_categories': missing_categories
            }
        else:
            print("❌ No transactions found in database")
            return {'total_transactions': 0}
            
    except Exception as e:
        logger.error(f"❌ Error during validation: {e}")
        print(f"❌ Error during validation: {e}")
        return {'error': str(e)}

def populate_budget_templates(conn):
    """Populate budget templates from imported transaction categories"""
    print("\n💰 POPULATING BUDGET TEMPLATES")
    print("-" * 40)
    
    cursor = conn.cursor()
    
    try:
        # Get unique categories from transactions
        cursor.execute("""
            SELECT DISTINCT category 
            FROM transactions 
            WHERE category IS NOT NULL AND category != ''
            ORDER BY category
        """)
        categories = cursor.fetchall()
        
        print(f"📂 Found {len(categories)} unique categories")
        
        # Insert budget templates
        created_count = 0
        for (category,) in categories:
            try:
                cursor.execute("""
                    INSERT OR IGNORE INTO budget_templates 
                    (category, budget_amount, notes, is_active, created_at, updated_at)
                    VALUES (?, 0.00, 'Auto-created from transaction data', 1, ?, ?)
                """, (category, datetime.utcnow().isoformat(), datetime.utcnow().isoformat()))
                
                if cursor.rowcount > 0:
                    created_count += 1
                    print(f"   ✅ Created budget template: {category}")
                    
            except Exception as e:
                logger.error(f"Error creating budget template for {category}: {e}")
        
        conn.commit()
        print(f"✅ Created {created_count} budget templates")
        
    except Exception as e:
        logger.error(f"❌ Error populating budget templates: {e}")
        print(f"❌ Error populating budget templates: {e}")

def generate_final_report():
    """Generate final import report"""
    print("\n📋 FINAL IMPORT REPORT")
    print("=" * 60)
    print(f"Import completed at: {datetime.now()}")
    print(f"Database backup: {backup_file}")
    print("\nImport Results:")
    if 'import_results' in locals():
        for table, results in import_results.items():
            print(f"  {table}: {results['imported']}/{results['total']} imported")
    
    print("\nNext steps:")
    print("1. Test the web application with imported data")
    print("2. Verify charts and analytics are working")
    print("3. Check budget categories are properly populated")
    print("4. Remove backup file if everything looks good")
    print("=" * 60)

# Run validation and final steps
validation_results = validate_import(conn)
populate_budget_templates(conn)
generate_final_report()


✅ IMPORT VALIDATION
----------------------------------------
📊 Total transactions imported: 6023
📅 Date range: 2022-01-03 to 2025-06-30
📂 Unique categories: 23
👥 Unique owners: 3
⚠️ Missing descriptions: 0
⚠️ Missing categories: 0

📋 Sample imported data (latest 5 transactions):
   2025-06-30 | Samba's Karina | $44 | Business
   2025-06-30 | venmo | $70 | Discretionary
   2025-06-30 | Girasoul | $-323.69 | Business
   2025-06-29 | Meta Publicity | $25 | Business
   2025-06-28 | HEB  | $53.8 | Living Expenses

💰 POPULATING BUDGET TEMPLATES
----------------------------------------
📂 Found 23 unique categories
   ✅ Created budget template: Argentina
   ✅ Created budget template: Beauty
   ✅ Created budget template: Business
   ✅ Created budget template: Charity
   ✅ Created budget template: Debt
   ✅ Created budget template: Dining Out
   ✅ Created budget template: Discretionary
   ✅ Created budget template: Dogs
   ✅ Created budget template: Gifts
   ✅ Created budget template: Governmen

C:\Users\ELCACAZ\AppData\Local\Temp\ipykernel_1348\10035739.py:95: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  """, (category, datetime.utcnow().isoformat(), datetime.utcnow().isoformat()))


In [15]:
# =============================================================================
# CLEANUP
# =============================================================================

def cleanup():
    """Clean up resources"""
    try:
        conn.close()
        logger.info("✅ Database connection closed")
        print("✅ Import process completed successfully")
    except Exception as e:
        logger.error(f"❌ Error during cleanup: {e}")

# Clean up
cleanup()

print("\n🎉 DATA IMPORT COMPLETED!")
print("Your personal finance database is now populated with all transaction data.")

2025-06-30 14:25:30,600 - INFO - ✅ Database connection closed


✅ Import process completed successfully

🎉 DATA IMPORT COMPLETED!
Your personal finance database is now populated with all transaction data.
